In [6]:
# import libraries

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import re #regular expression for searching for a word in a paragraph
from nltk.corpus import stopwords # to remove stopwords from data(doesnt give any values so remove it)
from nltk.stem.porter import PorterStemmer # used to give rootword for a particular word
from sklearn.feature_extraction.text import TfidfVectorizer # used to convert text into feature vectors
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [7]:
#download stopwords from nltk library
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
print(stopwords.words('English'))
# these words will be removed from the dataset

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [16]:
#load the dataset 

df=pd.read_csv('train.csv')
df.head(5)
# 1: Fake News
# 0: Real News

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [17]:
df.shape

(20800, 5)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [19]:
#Data Preprocessing

In [21]:
#check for duplicate rows
df.duplicated().sum()

0

In [20]:
#check gor null values
df.isna().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [23]:
#handling missing values
#replace the missing values by 'Null string'

df_new=df.fillna('')
df_new.isna().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [27]:
# combine the'author' and 'title' columns and use it for the predictions, why 'text' is not using here because of its large size and it will take large tym aslo for processing


df_new['content']=df_new['title']+' '+df_new['author']
df_new['content'].head(5)

0    House Dem Aide: We Didn’t Even See Comey’s Let...
1    FLYNN: Hillary Clinton, Big Woman on Campus - ...
2    Why the Truth Might Get You Fired Consortiumne...
3    15 Civilians Killed In Single US Airstrike Hav...
4    Iranian woman jailed for fictional unpublished...
Name: content, dtype: object

In [31]:
# segregating data into x and y

x=df_new.drop('label',axis=1)
y=df_new['label']
x.head(2)

,id,title,author,text,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,"FLYNN: Hillary Clinton, Big Woman on Campus - ..."


In [33]:
y.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [43]:
# stemming: The process of reducing a word to its rootword
#eg: (actor,acting,actress,etc) --> rootword is 'act'


port_stem=PorterStemmer()

def stemming(content):
    stemmed_content= re.sub('[^a-zA-Z]',' ',content)
    stemmed_content=stemmed_content.lower()
    stemmed_content=stemmed_content.split()
    stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content=' '.join(stemmed_content)
    return stemmed_content

In [44]:
# apply the defined function to 'content' column in dataset


df_new['content']=df_new['content'].apply(stemming)

In [45]:
print(df_new['content'])

0        hous dem aid even see comey letter jason chaff...
1        flynn hillari clinton big woman campu breitbar...
2                   truth might get fire consortiumnew com
3        civilian kill singl us airstrik identifi jessi...
4        iranian woman jail fiction unpublish stori wom...
                               ...                        
20795    rapper trump poster child white supremaci jero...
20796    n f l playoff schedul matchup odd new york tim...
20797    maci said receiv takeov approach hudson bay ne...
20798    nato russia hold parallel exercis balkan alex ...
20799                            keep f aliv david swanson
Name: content, Length: 20800, dtype: object


In [50]:
#Seperating data into x & y


x=df_new['content'].values
y=df_new['label'].values
print(x)
print(y)

['hous dem aid even see comey letter jason chaffetz tweet darrel lucu'
 'flynn hillari clinton big woman campu breitbart daniel j flynn'
 'truth might get fire consortiumnew com' ...
 'maci said receiv takeov approach hudson bay new york time michael j de la merc rachel abram'
 'nato russia hold parallel exercis balkan alex ansari'
 'keep f aliv david swanson']
[1 0 1 ... 0 1 1]


In [ ]:
#converting textual data to numerical data-then only machine can understand the textual data

vectorizer=TfidfVectorizer()
vectorizer.fit(x)
x=vectorizer.transform(x)

In [60]:
print(x)

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420

In [63]:
# splitting dataset into training and test data

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)
print("x_train: ",x_train.shape)
print("x_test: ",x_test.shape)
print("y_train: ",y_train.shape)
print("y_test: ",y_test.shape)

x_train:  (16640, 17128)
x_test:  (4160, 17128)
y_train:  (16640,)
y_test:  (4160,)


In [65]:
# training the model-Logistic regression

model=LogisticRegression()
model.fit(x_train,y_train)

LogisticRegression()

In [67]:
# model evaluation
#accuracy score on the training data

y_train_prediction=model.predict(x_train)
training_data_accuracy=accuracy_score(y_train_prediction,y_train)
print("Accuracy score for the training: ",training_data_accuracy *100)

Accuracy score for the training:  98.74399038461539


In [68]:
#accuracy score on the testing data

y_test_prediction=model.predict(x_test)
testing_data_accuracy=accuracy_score(y_test_prediction,y_test)
print("Accuracy score for the testing: ",testing_data_accuracy *100)

Accuracy score for the testing:  97.83653846153845


In [69]:
# Making a predictive system

x_new=x_test[0]

prediction=model.predict(x_new)
print(prediction)

if prediction ==0:
    print("The news is Real")
else:
    print("The news is Fake")

[0]
The news is Real
